In [1]:
from xml.etree import ElementTree as ET
import folium
import numpy as np
from shapely.geometry import Point, LineString
import pandas as pd
from sqlalchemy import create_engine
from math import radians, cos, sin, sqrt, atan2
import ast

Lógica para a execução do código
1. listar todos os carros que rodaram no dia
2. listar as linhas que estes carros rodaram
3. listar as rotas ociosas de cada linha
4. listar as rotas programadas de cada linha
5. listar a rota realizada pelo carro (dados mix)
6. fazer o comparativo entre realizada e programada
7. enviar email com os carros ofensores

Todos os dados estão no banco postgres.

In [2]:
dbname = 'poc-db'
user = 'adminpoc'
password = 'adminpoc'
host = 'localhost'
port = '5440' 

In [3]:
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')

In [4]:
ns = {'kml': 'http://www.opengis.net/kml/2.2'}

# Paths

In [5]:
planned_trip_path = '../notebooks/data/1.1 LINHAS RMTC [2023-11-29].kml'

# SQL

In [6]:
sql_geodata = """ 
select a.execution_date, a.registration, a.description, a.coordinates
    from hp.raw_geodata a
"""

In [7]:
sql_idle = """ 
select a.id, a.num_linha, a.descricao, b.direcao, b.pontos_geolocalizacao
from
    hp.raw_linhas a,
    hp.raw_trajetos b
where a.id = b.linha_id
"""

In [8]:
sql_schedule = """ 
SELECT id, "data", linha, carro, re, nome, dthr_saida, dthr_retorno, dthr_chegada
FROM hp.raw_escalas_programadas rep
"""

In [9]:
sql_routes = """
SELECT id, linha_id, direcao, pontos_geolocalizacao
FROM hp.raw_trajetos;
"""

# Dataframes

In [10]:
df_trip_made = pd.read_sql(sql_geodata, engine)

In [11]:
df_trip_idle = pd.read_sql(sql_idle, engine)

In [12]:
df_schedule = pd.read_sql(sql_schedule, engine)

In [13]:
df_routes = pd.read_sql(sql_routes, engine)

# Functions

In [14]:
def read_and_extract_placemarks(file_path, namespace):
    """ 
    :param: file_path: caminho para o arquivo kml
    :return: lista de dicionários com os nomes e coordenadas dos placemarks

    A função lê um arquivo kml e extrai os placemarks, retornando uma lista de dicionários com os nomes e coordenadas dos placemarks.
    """
    tree = ET.parse(file_path)
    root = tree.getroot()
    placemarks = []
    for placemark in root.findall('.//kml:Placemark', namespace):
        name = placemark.find('kml:name', namespace).text if placemark.find('kml:name', namespace) is not None else "Unnamed"
        coordinates = placemark.find('.//kml:coordinates', namespace).text.strip() if placemark.find('.//kml:coordinates', namespace) is not None else "No coordinates"
        placemarks.append({'name': name, 'coordinates': coordinates})
    return placemarks

In [15]:
def parse_coordinates(coordinates_str):
    """
    param: coordinates_str: string de coordenadas (lat, long)
    return: lista de tuplas de coordenadas (long, lat)

    A função recebe uma string de coordenadas (lat, long) e retorna uma lista de tuplas de coordenadas (long, lat).
    """
    coords = coordinates_str.split()
    return [(float(coord.split(',')[1]), float(coord.split(',')[0])) for coord in coords]

In [16]:
def add_placemarks_to_map(placemarks, map):
    """ 
    param: placemarks: lista de dicionários com as informações dos placemarks
    return: map: objeto do tipo folium.Map

    Quando o arquivo conter somente um ponto, por exemplo uma tupla de coordenadas,ele será plotado como um marcador de viagem realizada. Quando houver
    uma lista de coordenadas, oriundas de um arquivo com trajetória, será plotado uma linha entre os pontos.
    """
    for i, placemark in enumerate(placemarks):
        parsed_coords = parse_coordinates(placemark['coordinates'])
        if len(parsed_coords) == 1:  # Para um único ponto
            folium.Marker(
                location=parsed_coords[0],
                icon=folium.Icon(icon='fa-van-shuttle', prefix='fa',color='blue')
            ).add_to(map)

        else:  # Para múltiplos pontos (trajetória)
            if 'VOLTA' in placemark['name']:
                folium.PolyLine(locations=parsed_coords, color='green', weight=2.5, opacity=1, dash_array='5, 5').add_to(map)

            else:  # Para outros casos, usa uma linha contínua
                folium.PolyLine(locations=parsed_coords, color='red', weight=2.5, opacity=1).add_to(map)

In [17]:
def haversine(lon1, lat1, lon2, lat2):
    # Raio da Terra em quilômetros
    R = 6371.0

    # Conversão de coordenadas de graus para radianos
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Diferenças nas coordenadas
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Fórmula de Haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance

In [18]:
def extract_coords(coords_list):
    return [[coord['lat'], coord['lng']] for coord in coords_list]

# Transformações

# df_schedule

In [19]:
df_schedule.dtypes

id                       int64
data                    object
linha                   object
carro                    int64
re                      object
nome                    object
dthr_saida      datetime64[ns]
dthr_retorno    datetime64[ns]
dthr_chegada    datetime64[ns]
dtype: object

In [20]:
if df_schedule['data'].dtype == 'object':
    df_schedule['data'] = pd.to_datetime(df_schedule['data'])

## df_planned_trp

In [21]:
placemarks_planned = read_and_extract_placemarks(planned_trip_path, ns)

In [22]:
df_planned_trip = pd.DataFrame(placemarks_planned)

In [23]:
df_planned_trip['Código'] = df_planned_trip['name'].apply(lambda x: x.split('-', 1)[0])

In [24]:
df_planned_trip['identificacao'] = df_planned_trip['name'].apply(lambda x: x.split('-', 1)[1])
df_planned_trip['identificacao'] = df_planned_trip['identificacao'].apply(lambda x: x.split('- ', 1)[0])
df_planned_trip['identificacao'] = df_planned_trip['identificacao'].apply(lambda x: x.split(' ', 1)[0])

In [25]:
df_planned_trip['Código'] = df_planned_trip['name'].apply(lambda x: x.split('-', 1)[0])
df_planned_trip['Descrição e Sentido'] = df_planned_trip['name'].apply(lambda x: x.split(' - ', 1)[1] if ' - ' in x else '')
df_planned_trip['Descrição'] = df_planned_trip['Descrição e Sentido'].apply(lambda x: x.rsplit(' - ', 1)[0] if ' - ' in x else x)
df_planned_trip['Sentido'] = df_planned_trip['Descrição e Sentido'].apply(lambda x: x.rsplit(' - ', 1)[1] if ' - ' in x else '')
df_planned_trip.drop('Descrição e Sentido', axis=1, inplace=True)

In [26]:
de_para = {
    '1': 'A', '2': 'B', '3': 'C', '4': 'D', '5': 'E',
    '6': 'F', '7': 'G', '8': 'H', '9': 'I', '21': 'A',
    '22': 'B', '23': 'C', '24': 'D', '25': 'E', '26': 'F',
    '27': 'G', '28': 'H', '29': 'I', '31': 'A', '32': 'B',
    '33': 'C', '34': 'D', '35': 'E', '36': 'F', '37': 'G',
    '38': 'H', '39': 'I'
}

In [27]:
df_planned_trip['sublinha'] = df_planned_trip['identificacao'].apply(lambda x: de_para[x[:-1]])

In [28]:
df_planned_trip['concatenated'] = df_planned_trip['Código'].astype(str) + df_planned_trip['sublinha']

## df_trip_idle

In [29]:
df_trip_idle['num_linha_numerico'] = df_trip_idle['num_linha'].str.extract(r'^(\d+)')

In [30]:
df_trip_idle

,id,num_linha,descricao,direcao,pontos_geolocalizacao,num_linha_numerico
0,1,002A,PARQUE ATHENEU / T ISIDORIA / PAULO GARCIA,ida,"[{'lat': -16.76353, 'lng': -49.27021}, {'lat':...",002
1,1,002A,PARQUE ATHENEU / T ISIDORIA / PAULO GARCIA,volta,"[{'lat': -16.74154, 'lng': -49.19146}, {'lat':...",002
2,2,002B,PARQUE ATHENEU / T. ISIDORIA,ida,"[{'lat': -16.76353, 'lng': -49.27021}, {'lat':...",002
3,2,002B,PARQUE ATHENEU / T. ISIDORIA,volta,"[{'lat': -16.74154, 'lng': -49.19146}, {'lat':...",002
4,3,002C,PC TRINDADE / T ISIDORIA / PRACA CIVICA,ida,"[{'lat': -16.76353, 'lng': -49.27021}, {'lat':...",002
...,...,...,...,...,...,...
596,319,9901,COPA DA ESTRATEGIA - TURMA I,volta,"[{'lat': -16.705151223923, 'lng': -49.30067673...",9901
597,320,9902,COPA DA ESTRATEGIA - TURMA II,ida,"[{'lat': -16.65976138497988, 'lng': -49.261181...",9902
598,320,9902,COPA DA ESTRATEGIA - TURMA II,volta,"[{'lat': -16.705151223923, 'lng': -49.30067673...",9902
599,321,9903,COPA DA ESTRATEGIA - TURMA III,ida,"[{'lat': -16.65976138497988, 'lng': -49.261181...",9903


# Execution

* Lógica para a execução do código
* listar todos os carros que rodaram no dia
* listar as linhas que estes carros rodaram
* listar as rotas ociosas de cada linha
* listar as rotas programadas de cada linha
* listar a rota realizada pelo carro (dados mix)
* fazer o comparativo entre realizada e programada
* enviar email com os carros ofensores
* Todos os dados estão no banco postgres.

## Listando carro, motoristas e linhas que rodaram no dia

In [31]:
df_filtered = df_schedule[df_schedule['data'] == pd.Timestamp('2024-04-15')]

In [32]:
# Agrupar por 'carro' e agregar linhas e nomes únicos
df_report = df_filtered.groupby(['data','carro']).agg({
    'linha': lambda x: x.unique(),
    're': lambda x: x.unique(),
    'nome': lambda x: x.unique()
}).reset_index()


In [33]:
df_report

,data,carro,linha,re,nome
0,2024-04-15,1201,[117A],"[000007200, 000018995]","[LINDOMAR ANTONIO GARCIA, RICARDO ALEXANDRE CH..."
1,2024-04-15,20017,"[572A, 199A, 326A, 032A, 706B, 039A, 501A, 186...","[000014901, 000019061]","[DERLEY RICARDO DA SILVA, EDUARDO ROCHA DOS SA..."
2,2024-04-15,20018,"[015A, 176A, 160A, 400A]","[000019573, 000019710]","[EVANDRO SILVESTRE DA SILVA CUNHA, FERNANDO MA..."
3,2024-04-15,20019,"[915A, 401 , 400A]","[000019643, 000019504]","[RONILDO RIBEIRO ALVINO, UEMERSON FERREIRA DIAS]"
4,2024-04-15,20020,"[186A, 521A, 031A, 020A, 035A, 022A]","[000008922, 000017885]","[WALDIR ALVES DE ALMEIDA, LEANDRO TAVARES MEND..."
...,...,...,...,...,...
287,2024-04-15,20551,"[973A, 171A, 960B]","[000016195, 000019557]","[SIMAO GOMES DOS SANTOS, JOSE WENDERSON NEVES ..."
288,2024-04-15,20552,"[002A, 021A, 014A]","[000018638, 000017028]","[RAIMUNDO ALVES DE AGUIAR, BRAZ LUIZ DE PAULA]"
289,2024-04-15,20553,"[660A, 650A, 651A, 565A, 203B]","[000019296, 000018207]","[RODRIGO ALVES COELHO, WASHINGTON DOMINGOS DE ..."
290,2024-04-15,20805,"[025A, 025C]","[000014463, 000017814]","[DANIEL ALMEIDA DOS SANTOS, EDIMAR RODRIGUES R..."


## Adicionando as rotas ociosas ao dataframe

In [34]:
df_trip_idle

,id,num_linha,descricao,direcao,pontos_geolocalizacao,num_linha_numerico
0,1,002A,PARQUE ATHENEU / T ISIDORIA / PAULO GARCIA,ida,"[{'lat': -16.76353, 'lng': -49.27021}, {'lat':...",002
1,1,002A,PARQUE ATHENEU / T ISIDORIA / PAULO GARCIA,volta,"[{'lat': -16.74154, 'lng': -49.19146}, {'lat':...",002
2,2,002B,PARQUE ATHENEU / T. ISIDORIA,ida,"[{'lat': -16.76353, 'lng': -49.27021}, {'lat':...",002
3,2,002B,PARQUE ATHENEU / T. ISIDORIA,volta,"[{'lat': -16.74154, 'lng': -49.19146}, {'lat':...",002
4,3,002C,PC TRINDADE / T ISIDORIA / PRACA CIVICA,ida,"[{'lat': -16.76353, 'lng': -49.27021}, {'lat':...",002
...,...,...,...,...,...,...
596,319,9901,COPA DA ESTRATEGIA - TURMA I,volta,"[{'lat': -16.705151223923, 'lng': -49.30067673...",9901
597,320,9902,COPA DA ESTRATEGIA - TURMA II,ida,"[{'lat': -16.65976138497988, 'lng': -49.261181...",9902
598,320,9902,COPA DA ESTRATEGIA - TURMA II,volta,"[{'lat': -16.705151223923, 'lng': -49.30067673...",9902
599,321,9903,COPA DA ESTRATEGIA - TURMA III,ida,"[{'lat': -16.65976138497988, 'lng': -49.261181...",9903


In [35]:
# Expande a coluna 'linhas' para que cada elemento da lista se torne uma linha
df_report_exploded = df_report.explode('linha')
